In [ ]:
import numpy as np
import sklearn.ensemble
import torch
import pandas
import joblib

from matplotlib import pyplot as plt
from lib import model, glucose_dataset

# Introduction

This notebook provides a brief walkthrough of the public code release for our KDD 2018 paper: Deep Multi-Output Forecasting: Learning to Accurately Predict Blood Glucose Trajectories. The full paper is available via arXiv: https://arxiv.org/abs/1806.05357. We hope to release our glucose data to the general public soon. In the meantime, people interested in blood glucose forecasting may be interested in the recently released OhioT1DM dataset: http://smarthealth.cs.ohio.edu/OhioT1DM-dataset.html.

# Data
We have included both the processed and unprocessed dataset used to generate our results. This data was collected by authors Mamta Jaiswal, Dr. Lynn Ang, and Dr Rodica Pop-Busui.

## Unprocessed
The unprocessed dataset, data/unprocessed_cgm_data.xlsx, is an excel file with one sheet per recording session (from baseline to 36 months). Each row is one individual, note that patient ids are consistent across recording sessions, and not all patients have all recording sessions. The CGM data is giving at 5 minute resolution. The unprocessed data also contain information on the daily insulin dose and delivery method, which was not used in the paper. 

In [ ]:
unprocessed = pandas.read_excel('data/unprocessed_cgm_data.xlsx', sheet_name=None)

In [ ]:
unprocessed.keys()

In [ ]:
unprocessed['Baseline']

## Processed
The processed data is stored as four pickle files (accessible via joblib), data/processed_cgm_data_{train/validation/test}.pkl and data/processed_cgm_coeffs.pkl. To process we:

1. Remove data points which differ from previous ones by more than 40 mg/dL, as these measurements are almost certainly the result of sensor error
2. Impute small data gaps using linear interpolation.
3. Split data into contiguous chunks, splitting either on missing data or when a chunk is >101 measurements long
4. (PolyMO) compute coefficient bins on the training data.

The test set is constructed using the most recent session from each patient (approximately 10% of the data). 

We also include a differently processed version of the data, data/alternative_cgm_data_{train/test}, which we found useful for other projects. This data is constructed on a per-day basis, removing days with excessive missingness. Importantly, each day is linked to the ID of the patient it came from.

In [ ]:
data_tr = glucose_dat_train_rec = glucose_dataset.GlucoseDataset(data_pkl='data/processed_cgm_data_train.pkl',
                                                                 max_pad=101,
                                                                 output_len=6, # set 1 for Recursive, 6 for MO
                                                                 output_dim=361,
                                                                 polynomial=False,
                                                                 degree=2,
                                                                 range_low=0,
                                                                 range_high=100,
                                                                 coeff_file='/data2/ifox/glucose/data/training_coefficient_percentiles_ridge_alpha1_roc40.pkl')

In [ ]:
for x, y_index, y_real, lens in data_tr:
    break

In [ ]:
# The polynomial fitting takes a while (several minutes), but is only required once before training
data_tr_poly = glucose_dat_train_rec = glucose_dataset.GlucoseDataset(data_pkl='data/processed_cgm_data_train.pkl',
                                                                      max_pad=101,
                                                                      output_len=6,
                                                                      output_dim=361,
                                                                      polynomial=True,
                                                                      degree=2,
                                                                      range_low=0,
                                                                      range_high=100,
                                                                      coeff_file='/data2/ifox/glucose/data/training_coefficient_percentiles_ridge_alpha1_roc40.pkl')

In [ ]:
for x, poly_index, y_real, lens in data_tr_poly:
    break

# Models

Our paper considers 8 classes of models:

Shallow Baselines
* Extrapolation
* Recursive Random Forest
* Multi-Output Random Forest

Deep Baselines
* Recursive RNN
* Multi-Output RNN

Our Approaches
* Sequential Multi-Output RNN
* Polynomial Multi-Output RNN
* Polynomial Sequential Multi-Output RNN

We will walk through how we implemented, trained, and evaluated each model

## Shallow Baselines

### Extrapolation

This is a simple linear extrapolation baseline implemented via Numpy. We extrapolate using the last 30 minutes (6 samples as our data was sampled at 5 minute intervals) to predict 30 minutes into the future.

In [ ]:
data_tr = np.cumsum(np.random.randn(1000, 16), axis=1)
data_ts = np.cumsum(np.random.randn(100, 10), axis=1)

In [ ]:
n_input = 6
horizon = 6
degree = 1
extrap_pred = []
for i in range(len(data_ts)):
    coeffs = np.polynomial.polynomial.polyfit(x=np.arange(n_input), y=data_ts[i][-n_input:], deg=degree)
    extrap_pred.append(np.polyval(p=np.flip(coeffs, axis=0), x=np.arange(horizon)+n_input))

### Recursive and Multi-Output Random Forest

Implemented using scikit-learn. Note the scikit-learn implementation automatically infers output size during the fitting step. 

#### Recursive

In [ ]:
rf_rec = sklearn.ensemble.RandomForestRegressor(n_estimators=100, n_jobs=-1)

In [ ]:
# Note, for actually training recursive models, you should use all of the data by taking input_size tiles
X_rec_tr = data_tr[:, :10]
y_rec_tr = data_tr[:, 10:11].ravel()

In [ ]:
rf_rec.fit(X_rec_tr, y_rec_tr)

In [ ]:
# recursive prediction
X_mod = data_ts.copy()
p_rec_arr = []
for i in range(6):
    p = rf_rec.predict(X_mod)
    p_rec_arr.append(p.reshape(-1, 1))
    X_mod = np.concatenate((X_mod[:, 1:], p.reshape(-1, 1)), axis=1)

#### Multi-Output

In [ ]:
# Note, for actually training recursive models, you should use all of the data by taking input_size tiles
X_mo_tr = data_tr[:, :10]
y_mo_tr = data_tr[:, 10:]

In [ ]:
rf_mo = sklearn.ensemble.RandomForestRegressor(n_estimators=100, n_jobs=-1)

In [ ]:
rf_mo.fit(X_mo_tr, y_mo_tr)

In [ ]:
p_mo_arr = rf_mo.predict(data_ts)

## Deep Models
Our deep baselines are all implemented in PyTorch. They are a bit more involved to train. The basic training procedure is outlined in lib/trainer.py in the ExperimentTrainer class. The train_sup function is used to fit the provided model. The use of TensorboardX is not required, but convenient for monitoring losses. The data is assumed to be in the form of a pytorch dataset in the form of lib/glucose_dataset.py (though the specifics can vary greatly).

Note that the dataset code requires precomputed polynomial coefficients for the PolyMO setting. This can be done using Numpy's polyfit function on your training data. 

The cuda flag should be set to True if a GPU is available.

### Recursive Baseline

In [ ]:
rec_rnn = model.RecursiveRNN(input_dim=1, output_dim=361, hidden_size=512, depth=2,  cuda=False)

### Multi-Output Baseline

In [ ]:
mo_rnn = model.MultiOutputRNN(input_dim=1, output_dim=361, output_len=6, hidden_size=512, depth=2, cuda=False)

### Sequential Multi-Output

In [ ]:
seqmo_rnn = model.MultiOutputRNN(input_dim=1, output_dim=361, output_len=6, hidden_size=512, depth=2, cuda=False, sequence=True)

### Polynomial Multi-Output

In [ ]:
polymo_rnn = model.MultiOutputRNN(input_dim=1, output_dim=361, output_len=6, hidden_size=512, depth=2, cuda=False, polynomial=True, degree=1)

### Polynomial Sequential Multi-Output

In [ ]:
polymo_rnn = model.MultiOutputRNN(input_dim=1, output_dim=361, output_len=6, hidden_size=512, depth=2, cuda=False, sequence=True, polynomial=True, degree=1)